# LSTM


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

import tensorflow as tf
tf.get_logger().setLevel('CRITICAL')


C:\Users\feras\Anaconda3\envs\ForecastingRetailSales\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\feras\Anaconda3\envs\ForecastingRetailSales\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\feras\Anaconda3\envs\ForecastingRetailSales\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\feras\Anacond

In [2]:
import warnings
warnings.filterwarnings("ignore")

from datetime import timedelta, datetime
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import yaml
import mlflow
import tensorflow as tf
tf.__version__

from tqdm import tqdm

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, plotting, space_eval

from sklearn.model_selection import ParameterSampler
from scipy.stats.distributions import uniform, randint

from glob import glob
from tensorflow.keras.callbacks import EarlyStopping

# Get the current project path (where you open the notebook)
# and go up two levels to get the project path
current_dir = Path.cwd()
proj_path = current_dir.parent.parent


# make the code in src available to import in this notebook
import sys
sys.path.append(os.path.join(proj_path,'src'))

from metrics import *
from utils import *
from scalers import *

# Catalog contains all the paths related to datasets
with open(os.path.join(proj_path, 'conf/catalog.yml'), "r") as f:
    catalog = yaml.safe_load(f)['breakfast']
    
# Params contains all of the dataset creation parameters and model parameters
with open(os.path.join(proj_path, 'conf/params.yml'), "r") as f:
    params = yaml.safe_load(f)


In [3]:
def set_model(n_layers:int=1,init_units:int=50,n_unit_strategy:str='stable', 
              dropout_p:float=0.1, num_timesteps: int=8, num_series: int=5, 
              lr:float=0.001,optimizer:str='adam', loss:str='mape'):
    '''
    # Gives flexibility
    # Explore depth vs width of the RNN model
    https://stackoverflow.com/questions/59072728/what-is-the-rule-to-know-how-many-lstm-cells-and-how-many-units-in-each-lstm-cel
    # there are no rule of thumb, but here the decrease strategy will devide by i the number of units at each layer.
    
    Args
        num_timesteps: the number of lags in the dataframes
        num_series: the number of time series, specify 1 if univariate
        n_unit_strategy: two options, stable and decrease. decrease will decrease the  
    '''
    model = Sequential()
    if n_unit_strategy == 'stable':
        n_units = [init_units] * n_layers
    if n_unit_strategy == 'decrease':
        n_units = [max(int(init_units/i),1) for i in range(1,n_layers+1)]
    
    if n_layers > 1:
        model.add(LSTM(n_units[0], return_sequences=True, input_shape=(num_timesteps, num_series)))
        model.add(Dropout(dropout_p)) # Prevent overfitting
        for i in range(1, n_layers):
            model.add(LSTM(n_units[i], return_sequences=True))
            model.add(Dropout(dropout_p)) 
        model.add(LSTM(n_units[-1]))
        model.add(Dropout(dropout_p))
    
    else:
        model.add(LSTM(n_units[0], return_sequences=False, input_shape=(num_timesteps, num_series)))
        model.add(Dropout(dropout_p)) # Prevent overfitting

    model.add(Dense(units=1))
    
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr)
    
    model.compile(optimizer=opt, loss=loss)

    return model

In [4]:
def prepare_data(df, lag_units, list_features:list=['UNITS'], g_features:list=[], lag_g_units:int=4):
    """
    For every feature in the list of features, create lagged features
    for a specified number of lagged units. The lenght must be the
    same for all of the features. After creating the new features,
    it filters on those and adds them to a list that will be 
    reshaped in the correct format.
    
    We go from a dataframe version:
    
    UNITS    | FEATURE_A | FEATURE_B |   ...
    ---------|-----------|-----------|-------
    
    For every feature:
        
        Create the lagged features:

        UNITS    | FEATURE_A | FEATURE_A-lag-1 |   ...
        ---------|-----------|-----------------|-------  

        Filter on each lagged feature and inverse columns:

        FEATURE_A-lag-N | ... |  FEATURE_A-lag-1
        ----------------|-----|------------------
        
        Convert to numpy as reshape by adding a dimension
        (nrows, nlags) -> (nrows, nlags, 1)
        
    Concatenate each "feature-specific matrix" on the last dimension
    
    [(nrows, lagged-features-A, 1),
     (nrows, lagged-features-B, 1),
                  ...
     (nrows, lagged-features-X, 1)]
     
     Results in: 
     
     (nrows, nlags, N)
    
    
    Note: We need to specify the column 'UNITS' in the list of features.
    
    """
    
    # Initialize a list
    list_df = []
    
    for feature in (list_features+g_features):
        # create new features that will have the convention lag-<col_name>-<lagged unit>
        if feature in list_features:
            make_lag_features(df, lag_units, col_name=feature, prefix_name=f'lag-{feature}',inplace=True)
        if feature in g_features:
            make_lag_features(df, lag_g_units, col_name=feature, prefix_name=f'lag-{feature}',inplace=True)
        # filter columns that were just created
        # we are only appending lag something, so not actual features
        dummy = df.filter(like=f'lag-{feature}')
        # invert columns left to right (lag-1 ... lag-N) -> (lag-N ... lag-1)
        dummy = dummy[dummy.columns[::-1]]
        # add an extra dimension
        dummy = dummy.values.reshape(dummy.shape[0], dummy.shape[1], 1)
        # add the matrice to a list where it will get concatenated at the end
        list_df.append(dummy)
        
    

    # matrices in list_df must have shape (n_rows, n_cols, 1)
    # so we can concatenate on the last axis
    x_features = np.concatenate(list_df, axis=2)
    y_feature = np.array(df['UNITS'].values)

    return x_features, y_feature, df['WEEK_END_DATE'].values

In [5]:
stores = list(params['breakfast']['dataset']['store_ids'].keys())
upcs = list(params['breakfast']['dataset']['upc_ids'].keys())
store_upc_pairs = list(itertools.product(stores, upcs))
store_upc_pairs

[(2277, 1600027527),
 (2277, 3800031838),
 (2277, 1111009477),
 (2277, 7192100339),
 (389, 1600027527),
 (389, 3800031838),
 (389, 1111009477),
 (389, 7192100339),
 (25229, 1600027527),
 (25229, 3800031838),
 (25229, 1111009477),
 (25229, 7192100339)]

In [7]:
%%time

# Step 1: Read data
merged_data = pd.read_csv(os.path.join(proj_path, catalog['output_dir']['dir'],
                                       catalog['output_dir']['merged']))
merged_data['WEEK_END_DATE'] = pd.to_datetime(merged_data['WEEK_END_DATE'])
merged_data['WEEK_END_DATE'] = merged_data['WEEK_END_DATE'] + timedelta(days=3)

# Step 2: Create list of stores and upc pairs
stores = list(params['breakfast']['dataset']['store_ids'].keys())
upcs = list(params['breakfast']['dataset']['upc_ids'].keys())
store_upc_pairs = list(itertools.product(stores, upcs))

# Create date folds
date_ranges = make_dates(params['breakfast']['experiment_dates'])


for store_id, upc_id in store_upc_pairs: 
    print(f'Processing store {store_id} upc {upc_id}')
    
    create_folder(os.path.join(proj_path, 'mlruns'))
    mlflow.set_tracking_uri('../../mlruns')
    mlflow.set_experiment(f'{store_id}_{upc_id}')
    
    # Iterate over each period, unpack tuple in each variable.
    # in each of the period, we will find the best set of parameters,
    # which will represent the time-series cross validation methodology
    for _, train_start, train_end, valid_start, valid_end, test_start, test_end in date_ranges.itertuples():
        print(f'Processing range {str(train_start.date())} to {str(test_end.date())}')
        
        
        list_features = ['UNITS','VISITS','HHS','FEATURE','DISPLAY']
        # Filter data here
        # Prepare the dataset for one UPC and one store, currently no-scaling
        filtered_data = merged_data[(merged_data['STORE_NUM']==store_id) &
                                     (merged_data['UPC']==upc_id)][list_features+['WEEK_END_DATE']].copy()

        ws = params['lstm']['window_size']
        # ┌────────────────── GOOGLE TRENDS STARTS HERE ────────────────────┐
        # │
        # Get product category and state
        state = params['breakfast']['dataset']['store_ids'][store_id]
        cat = params['breakfast']['dataset']['upc_ids'][upc_id]

        # Do we need an epty glist ?
        glist = []
        # List comprehension to iteratively read in the correct directory the google
        # trends csv files that match the appropriate pattern of * (everything) in respective state
        # returns a list of pandas dataframes where after we concat on the column axis using the index_col
        # After that, we reset the index, to get the Week as a column and convert to datetime
        # Finaly, we add 6 days as the google trends hits represents the end of week and corresponds
        # to Breakfast At the Frat dates
        glist = [
            pd.read_csv(path, 
                        skiprows= 3, 
                        index_col= 'Week', 
                        names= ['Week', parse_name([str(proj_path), f'/data/03_external/{cat}/', '.csv'],path)]) 
            for path in glob(str(proj_path / f'data/03_external/{cat}/*_{state}.csv'))
        ]
        glist_us =  [
            pd.read_csv(path, 
                        skiprows= 3, 
                        index_col= 'Week', 
                        names= ['Week', parse_name([str(proj_path), f'/data/03_external/{cat}/', '.csv'],path)]) 
            for path in glob(str(proj_path / f'data/03_external/{cat}/*_US.csv'))
        ]
        # Cross Category
        glist_cc_state =  [
            pd.read_csv(path, 
                        skiprows= 3, 
                        index_col= 'Week', 
                        names= ['Week', parse_name([str(proj_path), f'/data/03_external/cross_category/', '.csv'],path)]) 
            for path in glob(str(proj_path / f'data/03_external/cross_category/*_{state}.csv'))
        ]
        glist_cc_us =  [
            pd.read_csv(path, 
                        skiprows= 3, 
                        index_col= 'Week', 
                        names= ['Week', parse_name([str(proj_path), f'/data/03_external/cross_category/', '.csv'],path)]) 
            for path in glob(str(proj_path / f'data/03_external/cross_category/*_US.csv'))
        ]
        # This is where you decide on whether to add those series, comment if you do not
        # The series should all have unique names, and is based on name of the file
        glist.extend(glist_us)
        glist.extend(glist_cc_state)
        glist.extend(glist_cc_us)

        # Combine all of the series into one, column-wise
        gdata = pd.concat(glist, axis=1).reset_index()
        gdata = fix_col_syntax(gdata)
        # reset index will always set the Week as the first column, so select after that date.
        gdata_feature_cols = gdata.columns[1:].to_list()
        gdata['Week'] = pd.to_datetime(gdata['Week'])
        gdata['Week'] = gdata['Week'] + timedelta(days=6)
        # Join on filtered_data
        filtered_data = filtered_data.merge(gdata, left_on='WEEK_END_DATE', right_on='Week', how='left').drop(columns='Week')
        # │ 
        # └────────────────────── GOOGLE TRENDS ENDS HERE ─────────────────────────┘
        
        # Prepare data, pass in all of the data, it will filter it
        data, label, dates = prepare_data(filtered_data, 
                                          ws, 
                                          list_features=list_features,
                                          g_features=gdata_feature_cols,
                                          lag_g_units= params['lstm']['gtrends_window_size'])
        
        # Because we are working with numpy now, it's easier to select the indexes
        # Will need date indexes 
        date_series = pd.Series(dates)        
        
        train_series_idx = date_series[(date_series>=train_start) &
                                       (date_series<=train_end)].index
        valid_series_idx = date_series[(date_series>=valid_start) &
                                       (date_series<=valid_end)].index
        test_series_idx = date_series[(date_series>=test_start) &
                                      (date_series<=test_end)].index
        train_data = data[train_series_idx].copy()
        train_label = label[train_series_idx].copy()
        valid_data = data[valid_series_idx].copy()
        valid_label = label[valid_series_idx].copy()
        test_data = data[test_series_idx].copy()
        test_label = label[test_series_idx].copy()   
        
        # The scaling of 0 to 1 creates issue with the loss function, as the minimum will be 0 and maximum 1. 
        # a bad solution was to drop the row that has 0, but doesn't seem to help. 
        # another solution is to log the values
        
        scalers = {}
        # The last two features are already bounded 0 to 1, we subtract 2 from the shape which represent 'FEATURE', 'DISPLAY'
        # for feature_col in range(train_data.shape[-1]-2):
        for feature_col in range(train_data.shape[-1]):
            scalers[feature_col] = NormalizeScalerDf()
            train_data[:,:,feature_col] = scalers[feature_col].fit_transform(train_data[:,:,feature_col])
            valid_data[:,:,feature_col] = scalers[feature_col].transform(valid_data[:,:,feature_col])
            test_data[:,:,feature_col] = scalers[feature_col].transform(test_data[:,:,feature_col])
        scalers['label'] = NormalizeScalerDf()
        train_label = scalers['label'].fit_transform(train_label)
        valid_label = scalers['label'].transform(valid_label)
        test_label = scalers['label'].transform(test_label)

        # Replace NaN's with 0's which are caused by the lagged values. This was done after the scaling to avoid issues
        train_data = np.nan_to_num(train_data)
        valid_data = np.nan_to_num(valid_data)
        test_data = np.nan_to_num(test_data)
        
        
        
        # Initialize some of the parameters
        ws = params['lstm']['window_size']
        dp = params['lstm']['dropout']
        units_strategy = params['lstm']['units_strategy']
        optimizers = params['lstm']['optimizers']
        losses = params['lstm']['loss']
        num_series = train_data.shape[2]   

        # Step 3: Define a random search for these parameters, for hyperparameter tuning
#         random_number_generator = np.random.RandomState(0) 
        param_grid = {'lr': uniform(loc=0.0001, scale=0.01),
                      'init_units': randint(low=5, high= 100),
                      'total_layers': randint(low=1, high=2)}
        param_list = list(ParameterSampler(param_grid, n_iter=params['lstm']['search_iter'], random_state=3))
        
        
        # Perform hyperparameter search
        res = []
        for param_dict in tqdm(param_list):
            lr = param_dict['lr']
            init_units = param_dict['init_units']
            tot_lay = param_dict['total_layers']
        
            # Explain here, stop when validation loss doesn't decrease for two steps/iterations
            earlyStop=EarlyStopping(monitor="val_loss", verbose=0, mode='min', patience=2)
            # Initialize the model and train
            lstm_model = set_model(n_layers=tot_lay, init_units=init_units,
                                   n_unit_strategy=units_strategy, dropout_p=dp,
                                   num_timesteps=ws, num_series=num_series,
                                   lr=lr, optimizer=optimizers, loss=losses)
            history = lstm_model.fit(train_data, train_label, validation_data=(valid_data, valid_label), 
                                     epochs=100, verbose=0, shuffle=False, batch_size=1, callbacks=[earlyStop])

            # _ = lstm_model.predict(np.expand_dims(valid_data[0],axis=0))[0]
            # There are many warnings when making predictions, thus I use a comprehension loop
            # val_predictions = np.array([lstm_model.predict(np.expand_dims(dd,axis=0))[0] for dd in valid_data])
            val_predictions = lstm_model.predict(valid_data)
            
            y_pred = scalers['label'].inverse_transform(val_predictions)
            y_true = scalers['label'].inverse_transform(valid_label)

            test_metrics = get_metrics(y_true, y_pred)
            
            val_mape = mean_absolute_percentage_error(y_true, y_pred)

            param_res = {'learning_rate':lr,
                         'init_units':init_units,
                         'n_layers':tot_lay,
                         'val_mape':val_mape}

            res.append(param_res)
        
        # Select the optimal parameters
        best_params = pd.DataFrame(res).sort_values(by='val_mape', ascending=True).iloc[0]

        lstm_model = set_model(n_layers=int(best_params['n_layers']),
                               init_units=best_params['init_units'],
                               n_unit_strategy=units_strategy,
                               dropout_p=dp,
                               num_timesteps=ws,
                               num_series=num_series,
                               lr=best_params['learning_rate'],
                               optimizer=optimizers,
                               loss=losses)

        earlyStop=EarlyStopping(monitor="val_loss", verbose=0, mode='min', patience=2)
        history = lstm_model.fit(train_data, train_label, validation_data=(valid_data, valid_label),
                                 epochs=100, verbose=0, shuffle=False, batch_size=1, callbacks=[earlyStop])

        test_predictions = lstm_model.predict(test_data) 
        
        y_pred = scalers['label'].inverse_transform(test_predictions)
        y_true = scalers['label'].inverse_transform(test_label)

        
        fdir = os.path.join(proj_path, catalog['results']['dir'], f'{str(test_end.date())}')
        fname = os.path.join(fdir, f'lstm_exp4_{store_id}_{upc_id}.csv')
        create_folder(fdir)

        pd.DataFrame({'y_true': y_true,
                      'y_pred': y_pred.flatten()}).to_csv(fname)
        
        with mlflow.start_run():
            mlflow.log_param('model','lstm')
            mlflow.log_param('experiment','exp4')
            mlflow.log_params(best_params)
            mlflow.log_param('fold_end',str(test_end.date()))
            mlflow.log_params(params['lstm'])
            mlflow.log_metrics(test_metrics)
            mlflow.log_artifact(fname)
            mlflow.log_params({'g_cat_state':True,
                               'g_cat_us':True,
                               'g_cc_state':True,
                               'g_cc_us':True})


Processing store 2277 upc 1600027527
Processing range 2009-01-17 to 2011-01-29


100%|██████████| 20/20 [02:09<00:00,  6.50s/it]


CPU times: user 3min 46s, sys: 59.2 s, total: 4min 45s
Wall time: 2min 20s
